In [1]:
import cv2, numpy as np
import time
import theano
import os
from collections import OrderedDict, defaultdict
import six.moves.cPickle as pkl
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten, LSTM, Lambda, merge
from keras.layers import Embedding
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D 
from keras.optimizers import SGD
from keras.layers import GRU, TimeDistributed, RepeatVector, Merge, TimeDistributedDense
import h5py
import json
from collections import Counter
import matplotlib.pyplot as plt
import keras.backend as K
from attention_lstm import AttentionLSTM
#import theano.tensor as T
#import skimage.transform

Using Theano backend.


In [2]:
MEAN_VALUES = np.array([104, 117, 123]).reshape((3,1,1))
SEQUENCE_LENGTH = 45
MAX_SENTENCE_LENGTH = SEQUENCE_LENGTH - 3 # 1 for image, 1 for start token, 1 for end token
BATCH_SIZE = 20
OUTPUT_DIM = 512
ANNOTATION_DIM = 512
WORD_DIM = 512
ANNOTATION_SIZE=196

In [3]:
d = []
with open('features/flickr30kann_1000_p3.pkl', 'rb') as f:
    d = pkl.load(f)
#pkl.dump(d, open('features/flickr30kann_1000_p3.pkl', 'wb'), protocol=pkl.HIGHEST_PROTOCOL)
vocab = d['vocab']
word_to_index = d['word_to_index']
index_to_word = d['index_to_word']

In [4]:
ins = d['INS']
gts = d['GTS']
ann_vec = d['AnnotationVectors']
ann_vec = ann_vec.reshape(1000,196,512)
z_mean = np.mean(ann_vec, axis=1)
print(ins.shape)
print(gts.shape)
print(ann_vec.shape)
print(z_mean.shape)
VOCAB_COUNT = len(vocab)

(1000, 44, 2452)
(1000, 44, 2452)
(1000, 196, 512)
(1000, 512)


In [10]:
ins_train = ins[1:900, :, :]
gts_train = gts[1:900, :, :]
ann_vec_train = ann_vec[1:900, :, :]
z_mean_train = z_mean[1:900, :]

ins_test = ins[900:, :, :]
gts_test = gts[900, :, :]
ann_vec_test = ann_vec[900:, :, :]
z_mean_test = z_mean[900:, :]

x_train = [ins_train, ann_vec_train, z_mean_train]
x_test = [ins_test, ann_vec_test, z_mean_test]

y_train = gts_train
y_test = gts_test
print(ins_train.shape)
print(ins_test.shape)
print(z_mean_train.shape)

(899, 44, 2452)
(100, 44, 2452)
(899, 512)


In [6]:
def build_model():
    print('Building Model')
    x_inp = Input(shape=(SEQUENCE_LENGTH-1, VOCAB_COUNT))
    z_inp = Input(shape=(ANNOTATION_SIZE, ANNOTATION_DIM,))
    z_mean = Input(shape=(ANNOTATION_DIM,))
    h_Dense = Dense(OUTPUT_DIM, input_dim=ANNOTATION_DIM, activation='softmax')(z_mean)
    c_Dense = Dense(OUTPUT_DIM, input_dim=ANNOTATION_DIM, activation='softmax')(z_mean)
    emb = Embedding(VOCAB_COUNT, WORD_DIM, input_length=SEQUENCE_LENGTH-1)(x_inp)
    tdemb = TimeDistributed(Dense(OUTPUT_DIM))(x_inp)
    print(h_Dense)
    print(c_Dense)
    #merge1 = merge([emb, h_Dense, c_Dense, z_inp], mode='concat', concat_axis=-1)
    aLstm = AttentionLSTM(output_dim=WORD_DIM, z_dim=ANNOTATION_DIM, return_sequences=True)([tdemb, h_Dense, c_Dense, z_inp])
    tdense = TimeDistributed(Dense(VOCAB_COUNT))(aLstm)
    act = Activation('softmax')(tdense)
    model = Model(input=[x_inp, z_inp, z_mean], output=act)
    '''
    model = Sequential()
    print('Adding Embedding')
    model.add(Embedding(VOCAB_COUNT, EMBEDDING_SIZE, input_length=SEQUENCE_LENGTH-1))
    print('Adding LSTM')
    model.add(AttentionLSTM(EMBEDDING_SIZE, Z_DIM, return_sequences=True))
    print('Adding TimeDistributed Dense')
    model.add(TimeDistributed(Dense(EMBEDDING_SIZE)))
    return model
    model.add(Dense(VOCAB_COUNT, activation='softmax'))
    '''
    #print(model.summary())
    return model

In [7]:
model = build_model()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Building Model
Softmax.0
Softmax.0
Elemwise{add,no_inplace}.0
Sum{axis=[1], acc_dtype=float64}.0
Shape.0
input shapes [(None, 44, 512), (None, 512), (None, 512), (None, 196, 512)]
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 44, 2452)      0                                            
____________________________________________________________________________________________________
input_3 (InputLayer)             (None, 512)           0                                            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 512)           262656      input_3[0][0]                    
____________________________________________________________________________________________________
dense_2 (Den

In [8]:
model.fit([ins, ann_vec, z_mean], gts, batch_size=1, nb_epoch=1, verbose=1)

Epoch 1/1
1000/1000 [==============================] - 99s - loss: 2.4658 - acc: 0.6252    


In [ ]:
model.fit(x_train, y_train, batch_size=20, nb_epoch=20, verbose=1)

In [ ]:
loss = model.evaluate(x_test, y_test, batch_size=20, verbose=1)

In [ ]:
y_pred = model.predict(x_test, batch_size=20, verbose=1)

In [8]:
def word_processing(dataset):
    allwords = Counter()
    for item in dataset:
        for sentence in item['sentences']:
            allwords.update(sentence['tokens'])
            
    vocab = [k for k, v in allwords.items()]
    vocab.insert(0, '<NULL>')
    vocab.append('<START>')
    vocab.append('<END>')
    vocab.append('<UNK>')

    word_to_index = {w: i for i, w in enumerate(vocab)}
    index_to_word = {i: w for i, w in enumerate(vocab)}
    return vocab, word_to_index, index_to_word

def import_flickr8kdataset():
    dataset = json.load(open('captions/dataset_flickr8k.json'))['images']
    #reduced length to a 300 for testing
    val_set = list(filter(lambda x: x['split'] == 'val', dataset))
    train_set = list(filter(lambda x: x['split'] == 'train', dataset))
    test_set = list(filter(lambda x: x['split'] == 'test', dataset))
    return train_set[:800]+val_set[:200]


In [9]:
def floatX(arr):
    return np.asarray(arr, dtype=theano.config.floatX)

#Prep Image uses an skimage transform
def prep_image(im):
    if len(im.shape) == 2:
        im = im[:, :, np.newaxis]
        im = np.repeat(im, 3, axis=2)
    # Resize so smallest dim = 224, preserving aspect ratio
    h, w, _ = im.shape
    if h < w:
        im = skimage.transform.resize(im, (224, w*224/h), preserve_range=True)
    else:
        im = skimage.transform.resize(im, (h*224/w, 224), preserve_range=True)

    # Central crop to 224x224
    h, w, _ = im.shape
    im = im[h//2-112:h//2+112, w//2-112:w//2+112]
    
    rawim = np.copy(im).astype('uint8')
    
    # Shuffle axes to c01
    im = np.swapaxes(np.swapaxes(im, 1, 2), 0, 1)
    
    # Convert to BGR
    im = im[::-1, :, :]

    im = im - MEAN_VALUES
    return rawim, floatX(im[np.newaxis])

In [10]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model


In [21]:
def language_model():
    model = Sequential()
    print('Adding Embedding')
    model.add(Embedding(VOCAB_COUNT, EMBEDDING_SIZE, input_length=SEQUENCE_LENGTH-1))
    print('Adding LSTM')
    model.add(LSTM(EMBEDDING_SIZE, return_sequences=False))
    #print('Adding TimeDistributed Dense')
    #model.add(TimeDistributed(Dense(EMBEDDING_SIZE)))
    return model

In [26]:
dataset = import_flickr8kdataset()
# Currently testing it out
dataset = [i for i in dataset[:100]]
vocab,word_to_index, index_to_word = word_processing(dataset)
#print vocab

In [27]:
def chunks(l, n):
    for i in xrange(0, len(l), n):
        yield l[i:i + n]

def process_images(dataset, coco=False, d_set="Flicker8k_Dataset"):
    ind_process = 1
    total = len(dataset)
    cnn_input = floatX(np.zeros((len(dataset), 3, 224, 224)))
    rawim_input = []
    sentences_tokens = []
    for i, image in enumerate(dataset):
        print "ind_process %s total %s" %(str(ind_process),str(total))
        ind_process+=1
        if coco:
            fn = './coco/{}/{}'.format(image['filepath'], image['filename'])
        else:
            fn = d_set+'/{}'.format(image['filename'])
        try:
            im = plt.imread(fn)
            rawim, cnn_input[i] = prep_image(im)
            sentences_tokens.append(image['sentences'][0]['tokens'])
            rawim_input.append(rawim)
        except IOError:
            continue
    return rawim_input, cnn_input, sentences_tokens


In [29]:
#rawim_array, cnnim_array, sentences_tokens = process_images(dataset, coco=False, d_set="Flicker8k_Dataset")
%matplotlib inline

In [28]:
def gen_image_partial_captions(images, captions, word_to_index, vocab_count, model):
    a_images = []
    a_captions = []
    next_words = []
    #vocab_size = len(vocab)
    for ind, image in enumerate(images):
        sentence = captions[ind]
        partial_caption_ar = np.zeros(SEQUENCE_LENGTH-1, dtype=np.int)
        
        words = ['<START>'] + sentence + ['<END>']
        assert len(words)<SEQUENCE_LENGTH
        for i in range(len(words) - 1):
            pc_copy = partial_caption_ar.copy()
            if words[i] in word_to_index:
                pc_copy[i] = word_to_index[words[i]]
            else:
                pc_copy[i] = word_to_index["<UNK>"]
            a_images.append(image)
            a_captions.append(pc_copy)
            #Generate next word output vector
            next_word = words[i + 1]
            if next_word in word_to_index:
                next_word_index = word_to_index[next_word]
            else:
                next_word_index = word_to_index["<UNK>"]
            next_word_ar = np.zeros(vocab_count, dtype=np.int)
            next_word_ar[next_word_index] = 1
            next_words.append(next_word_ar)
    v_i = np.array(a_images)
    v_c = np.array(a_captions)
    v_nw = np.array(next_words)
    return v_i, v_c, v_nw 

In [30]:
#vocab_count = len(word_to_index)
#print cnnim_array.shape
#v_i, v_c, v_nw = gen_image_partial_captions(cnnim_array, sentences_tokens, word_to_index, vocab_count)

In [31]:
VOCAB_COUNT = len(word_to_index)

In [32]:
def chunks(l, n):
    for i in xrange(0, len(l), n):
        yield l[i:i + n]

def process_cnn_features(dataset, model, coco=False, d_set="Flicker8k_Dataset"):
    ind_process = 1
    total = len(dataset)
    for chunk in chunks(dataset, 25):
        cnn_input = floatX(np.zeros((len(chunk), 3, 224, 224)))
        for i, image in enumerate(chunk):
            print "ind_process %s total %s" %(str(ind_process),str(total))
            ind_process+=1
            if coco:
                fn = './coco/{}/{}'.format(image['filepath'], image['filename'])
            else:
                fn = d_set+'/{}'.format(image['filename'])
            try:
                im = plt.imread(fn)
                _, cnn_input[i] = prep_image(im)
            except IOError:
                continue
        features = model.predict(cnn_input)
        print "Processing Features For Chunk"
        for i, image in enumerate(chunk):
            image['cnn features'] = features[i]
            
def process_captions_array(dataset):
    ind_process = 1
    total = len(dataset)
    for chunk in chunks(dataset, 25):
        for i in chunk:
            
            image["captions"]

In [33]:
model = VGG_16('weights/vgg16_weights.h5')
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')


In [ ]:
def predict(model, images, index_to_word, word_to_index):
    for image in images:
        caption = np.zeros(SEQUENCE_LENGTH - 1).reshape(1, SEQUENCE_LENGTH - 1)
        print(caption.shape)
        caption[0,0] = word_to_index["<START>"]
        count=0
        sentence = []
        a = image.reshape(1,3,224,224)
        #a = np.array([image])
        while True:
            out = model.predict([a, caption])
            index = out.argmax(-1)
            index = index[0]
            word = index_to_word[index]
            sentence.append(word)
            count+= 1
            if count >= SEQUENCE_LENGTH - 1 or index == word_to_index["<END>"]: #max caption length reach of '<eos>' encountered
                break
            caption[0,count] = index
        sent_str = " ".join(sentence)
        print("The Oracle says : %s" %sent_str)


In [ ]:
cnnim_list = []
for i in cnnim_array:
    cnnim_list.append(i)

In [ ]:
predict(model, cnnim_list, index_to_word, word_to_index)

In [ ]:
caption = np.zeros(SEQUENCE_LENGTH).reshape(1, SEQUENCE_LENGTH)
caption[0] = word_to_index["#START#"]
print cnnim_array[0].shape
t = np.array(cnnim_array[0])
print t.shape
out = model.predict([v_i, v_c])
print out
